In [2]:
import geopandas as gpd
import pandas as pd
import folium
import random
import shapely
import xyzservices.providers as xyz
from folium import plugins
import os
import branca

In [3]:
gdf = gpd.read_file('./locations.geojsonl.json').rename(columns={'fid':'photo_id'})

gdf = gdf.to_crs('EPSG:3310')

In [8]:
#Add the Stadia Maps Stamen Toner provider details via xyzservices
tile_provider = xyz.Stadia.StamenTonerLite

#Update the URL to include the API key placeholder
# tile_provider["url"] = tile_provider["url"] + f"?api_key={os.environ['STADIA_API_KEY']}"

#(api_key=os.environ["STADIA_API_KEY"])

#Create the folium TileLayer, specifying the API key
stamen = folium.TileLayer(
    tiles=tile_provider.build_url(),
    attr=tile_provider.attribution,
    name=tile_provider.name,
    max_zoom=tile_provider.max_zoom,
    detect_retina=True
)

In [9]:
def buffer_map(gdf, col, cmap):
    gdf = gdf.copy()
    gdf.geometry = gdf.buffer(15)
    m = gdf.explore(column = col, cmap = cmap,
    tiles = stamen,
    style_kwds={'fillOpacity': 1})
    plugins.Fullscreen(
    position="topright",
    title="Expand me",
    title_cancel="Exit me",
    force_separate_button=True,
).add_to(m)

    return m

In [10]:
m_all = buffer_map(gdf, 'points', branca.colormap.step.Dark2_04.scale(vmax=4, vmin=1))

In [11]:
m_all.save('map_all.html')

In [12]:
folium.__version__

'0.19.5'

## Team Results

In [13]:
teams = [f'team_{i}' for i in range(1,5)]

In [14]:
teams

['team_1', 'team_2', 'team_3', 'team_4']

In [15]:
def read_team(path, team):
    cols = ['team_id', 'photo_id', 'timestamp']
    df = pd.read_excel(path, sheet_name=team)[cols]
    return df.query('photo_id != 99')

In [16]:
read_team('./photo_scoring.xlsx', 'team_1')

,team_id,photo_id,timestamp
0,1,11,13:54:08.739000
1,1,1,13:54:08.739000
2,1,34,14:48:23.207000


In [17]:
all_teams = pd.concat([read_team('./photo_scoring.xlsx', team) for team in teams])

### nobody found these

In [18]:
buffer_map(gdf.query('~ photo_id.isin(@all_teams.photo_id)'), 'points', branca.colormap.step.Dark2_04.scale(vmax=4, vmin=1)).save('nobody.html')

### popular spots

In [19]:
popular = (all_teams.groupby('photo_id')
 .count()
 .reset_index()
 .rename(columns={'team_id':'team_count'})
 .sort_values('team_count', ascending=False)
 [['photo_id', 'team_count']])

In [20]:
buffer_map(gdf.merge(popular, on=['photo_id']), 'team_count', branca.colormap.step.YlOrRd_04.scale(vmax=4, vmin=1)).save('popular.html')

## all_teams

In [21]:
teams_merged = gdf.merge(all_teams, on='photo_id')

In [22]:
random.random()

0.9357232612362972

In [23]:
def jitter(row):
    
    max_jitter = 20
    chs_x = 1 if random.random() < 0.5 else -1
    chs_y = 1 if random.random() < 0.5 else -1
    new_x = row.geometry.x + (max_jitter*random.random() * chs_x)
    new_y = row.geometry.y + (max_jitter*random.random() * chs_y)
    row.geometry = shapely.Point(new_x, new_y)

    return row

In [24]:
buffer_map(teams_merged.apply(jitter, axis=1), 'team_id',
           branca.colormap.step.Spectral_04.scale(vmax=4, vmin=1)).save('all_teams.html')